> **Date:** 03/09/23
# Speach to text model

#### Goal:
Find a model for speach to text generation.  


#### Resources:
- Creating YouTube Captions with Wav2Vec [Link Colab](https://colab.research.google.com/github/Muennighoff/ytclipcc/blob/main/wav2vec_youtube_captions.ipynb)

## Installing dependencies

In [ ]:
!pip install transformers
!pip install torch

### Imports

In [ ]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from IPython.display import Audio
import moviepy.editor as mp
import torch
import librosa
import os

### Load models

In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

### Extract Audio

In [ ]:
clip = mp.VideoFileClip("clip.mp4")
end = min(clip.duration, end)

# Save the paths for later
clip_paths = []

# Extract Audio-only from mp4
for i in range(start, int(end), 10):
  sub_end = min(i+10, end)
  sub_clip = clip.subclip(i,sub_end)

  sub_clip.audio.write_audiofile("audio_" + str(i) + ".mp3")
  clip_paths.append("audio_" + str(i) + ".mp3")

### Transcribe Audio

In [ ]:
cc = ""

for path in clip_paths:
    # Load the audio with the librosa library
    input_audio, _ = librosa.load(path, 
                                sr=16000)

    # Tokenize the audio
    input_values = tokenizer(input_audio, return_tensors="pt", padding="longest").input_values

    # Feed it through Wav2Vec & choose the most probable tokens
    with torch.no_grad():
      logits = model(input_values).logits
      predicted_ids = torch.argmax(logits, dim=-1)

    # Decode & add to our caption string
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    cc += transcription + " "

# Here's your caption!
# Note that there may be mistakes especially if the audio is noisy or there are uncommon words
# If you picked the default video and change start to 0, you will see that the model gets confused by the word "Anakin"
print(cc)